# Data Cleaning 4: Untagged PER Entities

Strategy 1: Exact Matches

Strategy 2: Fuzzy Matches

In [4]:
from IPython.display import clear_output
import spacy
from spacy import displacy

import pickle

In [2]:
import pandas as pd

In [5]:
file = open("Files_Cleaning3/NER_data_Cleaned3.p", 'rb')
docs = pickle.load(file)

# Exact Matching to Confirmed Names

Let's generate a list of PER entity strings:

In [143]:
peoplestrings =[]
for doc, context in docs:
    for ent in doc.ents:
        if ent.label_=='PER':
            peoplestrings.append(ent.text)         

In [144]:
len(peoplestrings)

32339

In [145]:
peoplestrings = [string.lower() for string in peoplestrings]

In [146]:
len(set(peoplestrings))

12805

In [147]:
peoplestrings = list(set(peoplestrings))

In [11]:
untaggedtext

' Pago de'

In [12]:
strlist

['fernando', 'de', 'ledesma']

In [13]:
untaggedtokens

['Pago', 'de']

In [14]:
untaggedtokens[start+b]

'de'

In [15]:
b

0

In [16]:
span

''

In [ ]:
entry

In [ ]:
untaggedPERS = []

for doc, context in docs:
    ents = list(doc.ents)
    enttexts = []
    for ent in ents:
        enttexts.append(ent.text)
        untaggedtext = doc.text.replace(ent.text," ",1) #replace the entity span once with a blank
    for string in peoplestrings:
        strlist = string.split()
        try:
            untaggedtokens= untaggedtext.split()
            for i in range(len(untaggedtokens)):
                start = i
                b=0
                while (b<len(strlist)) and (untaggedtokens[start+b].lower() == strlist[b].lower()):
                    b= b+1
        except:
            pass
        end = start + b
        span = ' '.join(untaggedtokens[start:end])
        if (b !=0) and (span.lower() == string.lower()):
            docid = context['id']
            entry = [docid, string, span]
            untaggedPERS.append(entry)
            b=0
    print(context['id'])
    clear_output(wait=True)
        
#        if (string.lower() in doc.text.lower()) and (string.lower() not in enttexts):
 #           string = str(string)
  #          docid = context['id']
   #         entry = [docid, string]
    #        untaggedPERS.append(entry)

In [148]:
untaggedPERS = []

for doc, context in docs:
    untaggedtext = ''
    for token in doc:
        if token.ent_iob==2:
            untaggedtext = untaggedtext + " " + token.text
    for string in peoplestrings:
        b=0
        strlist = string.split()
        try:
            untaggedtokens= untaggedtext.split()
            for i in range(len(untaggedtokens)):
                start = i
                b=0
                while (b<=len(strlist)) and (untaggedtokens[start+b].lower() == strlist[b].lower()):
                    b= b+1
        except:
            pass
        end = start + b
        span = ' '.join(untaggedtokens[start:end])
        if (b !=0) and (len(range(start,end)) == len(strlist)):
            docid = context['id']
            entry = [docid, string, span, start, end]
            untaggedPERS.append(entry)
    print(context['id'])
    clear_output(wait=True)

8607


In [40]:
untaggedtokens[start:end]

[]

In [29]:
range(len(untaggedtokens))

range(0, 2)

In [27]:
span

''

In [17]:
strlist

['fernando', 'de', 'ledesma']

In [18]:
len(strlist)

3

In [ ]:
len(docs)

In [38]:
docs[99][1]

{'id': 100}

In [39]:
displacy.render(docs[99][0], style ='ent')

In [ ]:
#Need to remove duplicates with capitalization variations. Need to make sure spans are not double counted.

In [149]:
len(untaggedPERS)

2476

In [150]:
untaggedPERS_df = pd.DataFrame(untaggedPERS, columns = ['docid','match','string','start','end'])

In [151]:
untaggedPERS_df

,docid,match,string,start,end
0,2,costa,costa,62,63
1,52,costa,costa,1030,1031
2,55,costa,costa,1004,1005
3,82,costa,costa,269,270
4,85,juan,juan,11,12
...,...,...,...,...,...
2471,8513,juan de la,Juan de la,3,6
2472,8513,juan de la palma,Juan de la Palma,3,7
2473,8536,corral,corral,36,37
2474,8552,miguel,Miguel,687,688


In [152]:
untaggedPERS_df.to_csv('Files_Cleaning4/untaggedPERstrings_strat1.csv')

In [153]:
untaggedPERS_df[untaggedPERS_df['string']!='de']

,docid,match,string,start,end
0,2,costa,costa,62,63
1,52,costa,costa,1030,1031
2,55,costa,costa,1004,1005
3,82,costa,costa,269,270
4,85,juan,juan,11,12
...,...,...,...,...,...
2471,8513,juan de la,Juan de la,3,6
2472,8513,juan de la palma,Juan de la Palma,3,7
2473,8536,corral,corral,36,37
2474,8552,miguel,Miguel,687,688


In [154]:
untaggedPERstrings = pd.DataFrame(untaggedPERS_df['string'].value_counts())

In [155]:
untaggedPERstrings.to_csv('Files_Cleaning4/untaggedPERstrings_strat1_unique.csv')

In [156]:
untaggedPERstrings[untaggedPERstrings.index.str.contains(' ')]

,string
juan bautista,35
señor prouisor,24
juan baptista,17
Juan Bautista,15
francisco de,13
...,...
geronimo belasques,1
justa rufina,1
Geronima de çamudio,1
Diego de Alcala,1


Looking at the contents of this DataFrame in the .csv file, we have our work cut out for us. But we can remove matches with the strings "de", "costa", and "de la".

In [157]:
untaggedPERS_df = untaggedPERS_df[(untaggedPERS_df['string']!='de') & (untaggedPERS_df['string']!='de la') & (untaggedPERS_df['string']!='costa')]

In [158]:
len(untaggedPERS_df)

2042

In [160]:
untaggedPERS_df['addentity']=''
untaggedPERS_df['entstart']=''
untaggedPERS_df['entend']=''
untaggedPERS_df['label']=''

/Users/Felipe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/Felipe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/Felipe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

In [161]:
untaggedPERS_df.head()

,docid,match,string,start,end,addentity,label,entstart,entend
4,85,juan,juan,11,12,,,,
5,85,juan\n,juan,11,12,,,,
6,85,juan de,juan de,11,13,,,,
7,85,señor prouisor,señor prouisor,23,25,,,,
8,85,juan de la,juan de la,11,14,,,,


Some potential entities have matched multiple entity strings. We need to take the longest spans and keep those. To do this, we sort on docid, start, and end, in descending order, so the longest spans are first (ascending = False). Then, we use  df.drop_duplicates with keep =  first.

In [176]:
untaggedPERS_df = untaggedPERS_df.sort_values(['docid','start','end'], ascending = False).drop_duplicates(['docid','start'], keep = 'first')

In [177]:
untaggedPERS_df

,docid,match,string,start,end,addentity,label,entstart,entend
2475,8562,rubio,rubio,33,34,,,,
2474,8552,miguel,Miguel,687,688,,,,
2473,8536,corral,corral,36,37,,,,
2472,8513,juan de la palma,Juan de la Palma,3,7,,,,
2466,8507,bautista,Bautista,73,74,,,,
...,...,...,...,...,...,...,...,...,...
15,100,maria,maria,82,83,,,,
12,95,ana,ana,55,56,,,,
11,93,roque,roque,48,49,,,,
7,85,señor prouisor,señor prouisor,23,25,,,,


That leaves 1,624 potentially untagged PER entities to look at. 

In [178]:
untaggedPERS_df.to_csv('Files_Cleaning4/untaggedPERstrings_strat1_clean.csv')

In [2]:
import pandas as pd
untaggedPERS_df = pd.read_csv('Files_Cleaning4/untaggedPERstrings_strat1_manual.csv', index_col=0)

In [5]:
untaggedPERS_df = untaggedPERS_df.fillna('')

In [12]:
count = 0
for (index, row) in untaggedPERS_df.iterrows():
    if  (row['addentity']==''):
        count=count+1
        print(count)
        print(row['string'], row['match'])
        docid = row['docid']
        if ' ' in row['string']:
            strlist = row['string'].split()
        else: 
            strlist = [row['string']]
        for doc, context in docs:
            if context['id'] == docid: 
                for token in doc:
                    b=0
                    if token.ent_iob == 2 and token.text.lower()==strlist[0].lower():
                        start = token.i
                        while (b<len(strlist)) and (str(doc[start+b].text).lower() == strlist[b].lower()):
                            b= b+1
                        end = start + b
                        span = doc[start:end]
                        print(span)
                        if (b !=0) and (len(range(start,end)) == len(strlist)):
                            try:
                                displacy.render(doc[start-5:end+5],style='ent',jupyter=True)
                            except:
                                displacy.render(doc[start:end+5],style='ent',jupyter=True)
                            untaggedPERS_df.loc[index, 'addentity']=input('Add?')
                        clear_output(wait=True)

1
francisco de francisco de
francisco


Note: in this notebook I am only marking PER entities as 1. Some of these entities are clearly LOC and ORG entities, but these will be better found with LOC and ORG strings. 

In [3]:
untaggedPERS_df.groupby('addentity').count()

,Unnamed: 0.1,docid,match,string,start,end,label,entstart,entend
addentity,,,,,,,,,
0,1263,1263,1263,1263,1263,1263,0,0,0
1,342,342,342,342,342,342,0,0,0
len,18,18,18,18,18,18,0,0,0


In [10]:
untaggedPERS_df.to_csv('Files_Cleaning4/untaggedPERstrings_strat1_manual.csv')

In [ ]:
# Now make the edits (go through all value 1, add spans to newstart newend because some need edits)

In [6]:
untaggedPERS_df['newstart']=''
untaggedPERS_df['newend']=''

In [22]:
# untaggedPERS_df[untaggedPERS_df['addentity']=='e']='1'

In [7]:
count = 0
for (index, row) in untaggedPERS_df.iterrows():
    if (row['addentity']=='1'):
        docid = row['docid']
        if ' ' in row['string']:
            strlist = row['string'].split()
        else: 
            strlist = [row['string']]
        for doc, context in docs:
            if context['id'] == docid: 
                for token in doc:
                    b=0
                    if token.ent_iob == 2 and token.text.lower()==strlist[0].lower():
                        count=count+1
                        print(count)
                        start = token.i
                        while (b<len(strlist)) and (str(doc[start+b].text).lower() == strlist[b].lower()):
                            b= b+1
                        end = start + b
                        span = doc[start:end]
                        print(start, end, span)
                        if len(strlist)==len(range(start,end)):
                            try:
                                displacy.render(doc[start-5:end+5],style='ent',jupyter=True)
                            except:
                                displacy.render(doc[start:end+5],style='ent',jupyter=True)
                            c = input('See more?')
                            if c == 'y':
                                displacy.render(doc, style='ent', jupyter=True)
                            a=input('New start?')
                            b=input('New end?')
                            d= input('Add?')           
                            if d =='e':
                                untaggedPERS_df.loc[index,'addentity']='e'
                                if a !='':
                                    untaggedPERS_df.loc[index, 'newstart']=a
                                if b !='':
                                    untaggedPERS_df.loc[index,'newend']=b
                            else:
                                untaggedPERS_df.loc[index,'addentity']=d
                    
                            clear_output(wait=True)

128
36 38 señor prouisor


KeyboardInterrupt: 

In [8]:
untaggedPERS_df.groupby('addentity').count()

,Unnamed: 0.1,docid,match,string,start,end,label,entstart,entend,newstart,newend
addentity,,,,,,,,,,,
0,1272,1272,1272,1272,1272,1272,0,0,0,1272,1272
1,287,287,287,287,287,287,0,0,0,287,287
e,46,46,46,46,46,46,0,0,0,46,46
len,18,18,18,18,18,18,0,0,0,18,18


In [9]:
untaggedPERS_df.to_csv('Files_Cleaning4/untaggedPERstrings_strat1_manual.csv')